In [1]:
import sys
import json
import numpy as np
import pandas as pd

sys.path.append("..")

from graphstructure.simple_substructure import SubstructureSimple
from graphstructure.metrics import MetricFunction, REAL_LIFE_LOADS

In [2]:
REAL_LIFE_LOADS["plausibility"] = ["FAMD GMM", "FAMD Mahalanobis", "one class SVM", "FAMD one class SVM", "kde",
 "isolation forest", "local outlier factor"]  

REAL_LIFE_LOADS["plausibility"] = ["one class SVM"]          
nus = [0.25, 0.75, 0.9]

In [3]:
import math
def test_all(designs, metrics):
    objs = [] 
    evaluated_designs = []
    for d in designs:
        sub = SubstructureSimple.from_dict(d)
        obj_values = np.array(metrics(sub)[0])
        if any([math.isinf(x) for x in obj_values]):
            print('This design cant be tested skipping:', d)
            continue
            # print('This design cant be tested in ops and thus got scores of 0.0:', d)
            # obj_values[np.where(obj_values == 0.0)] = np.inf
        objs.append(obj_values)
        evaluated_designs.append(d)
    return objs, evaluated_designs

In [4]:
def to_df(objs, metrics, plaus_only=False):
    # print(metrics.metric_names)
    df_ret = pd.DataFrame(objs, columns=metrics.metric_names)
    metric_names_array = np.array(metrics.metric_names)
    if plaus_only:
        matching_instances = [x for x in metric_names_array if "one class svm" in x.lower()][0]
        print(matching_instances)
        return df_ret[matching_instances]
    return df_ret

In [5]:
import os
def evaluate(designs, file_path, out_dir, nus):
    dfs = []
    for nu in nus:
        metrics = MetricFunction.from_loads(REAL_LIFE_LOADS, nu=nu)
        objs, evaluated_designs = test_all(designs, metrics)
        if len(dfs) > 0:
            df = to_df(objs, metrics, plaus_only=True)
        else:
            df = to_df(objs, metrics)
        dfs.append(df)
    df = pd.concat(dfs, axis=1)
    print(df)
    data_merged = [{**d, **df.loc[i]} for i, d in enumerate(evaluated_designs)]
    filename = os.path.splitext(os.path.basename(file_path))[0]
    output_filename = out_dir + filename +"_evaluated.json"
    with open(output_filename, "w") as json_file:
        json.dump(data_merged, json_file, indent=4)
    print("Saved file", output_filename, "total designs", len(evaluated_designs))    

In [6]:
out_dir = "data/evaluated/"
in_dir = "data/to_evaluate/"
files = os.listdir(in_dir)

for file_ in files:
    with open(in_dir + file_) as f:
        synthetic = json.load(f)
        evaluate(synthetic, file_, out_dir, nus)


One Class SVM gamma=scale nu=0.25
Running with objectives: ['Cost', 'plausibility (One Class SVM) gamma=scale nu=0.25', 'wave', 'combined', 'compression', 'tipover', 'torque']
One Class SVM gamma=scale nu=0.75
Running with objectives: ['Cost', 'plausibility (One Class SVM) gamma=scale nu=0.75', 'wave', 'combined', 'compression', 'tipover', 'torque']
plausibility (One Class SVM) gamma=scale nu=0.75
One Class SVM gamma=scale nu=1


ValueError: The dual coefficients or intercepts are not finite. The input data may contain large values and need to be preprocessed.